In [1]:
import urllib.request
import pandas as pd
import boto3

## Função para coletar os dados

In [ ]:
def extract_data(url, filename):
    try:
        urllib.request.urlretrieve(url, filename)
    except Exception as e:
        print(e)       

In [ ]:
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/c9509ab4-6f6d-4b97-979a-0cf2a10c922b/download/311_service_requests_2015.csv", "data/dados_2015.csv")
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/b7ea6b1b-3ca4-4c5b-9713-6dc1db52379a/download/311_service_requests_2016.csv", "data/dados_2016.csv")
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/30022137-709d-465e-baae-ca155b51927d/download/311_service_requests_2017.csv", "data/dados_2017.csv")
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/311_service_requests_2018.csv", "data/dados_2018.csv")
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/311_service_requests_2019.csv", "data/dados_2019.csv")
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/script_105774672_20210108153400_combine.csv", "data/dados_2020.csv")


In [2]:
arquivos = [
    "data/dados_2015.csv",
    "data/dados_2016.csv",
    "data/dados_2017.csv",
    "data/dados_2018.csv",
    "data/dados_2019.csv",
    "data/dados_2020.csv",
]

dfs = {}

In [3]:
for arquivo in arquivos:
    ano = arquivo.split("_")[-1].split(".")[0]
    dfs[ano] = pd.read_csv(arquivo)

## Boto 

In [4]:

import os
from dotenv import load_dotenv

load_dotenv('*******')

aws_access_key_id = os.getenv('aws_access_key_id')
aws_secret_access_key = os.getenv('aws_secret_access_key')
region_name = os.getenv('region_name')

In [5]:
boto3.setup_default_session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)

s3 = boto3.client("s3")


In [6]:
content = "Olá S3"

with open("ola-s3.txt","w+") as f:
    f.write(content)

In [7]:
# arquivo, bucket, destino
s3.upload_file("ola-s3.txt","estudo-boston","bronze/ola-s3.txt")

# Parquet

In [8]:
from io import BytesIO

for ano, df in dfs.items():
    parquet_buffer = BytesIO()
    df.to_parquet(parquet_buffer)
    s3.put_object(
        Bucket="estudo-boston",
        Key=f"bronze/dados_{ano}.parquet",
        Body=parquet_buffer.getvalue(),
    )

In [9]:
response = s3.list_objects(Bucket="estudo-boston")

In [ ]:
keys = [obj['Key'] for obj in response ["Contents"]]

In [12]:
for i in range(len(keys)):
    print(keys[i])

bronze/
bronze/dados_2015.parquet
bronze/dados_2016.parquet
bronze/dados_2017.parquet
bronze/dados_2018.parquet
bronze/dados_2019.parquet
bronze/dados_2020.parquet
bronze/ola-s3.txt
silver/
